In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import hashlib
from itertools import compress
import graphviz
from IPython.display import Image  
from sklearn.externals.six import StringIO 
import pydotplus

In [2]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import ensemble
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

In [3]:
test_db = pd.read_csv('csvdir/test.csv')
train_db = pd.read_csv('csvdir/train.csv')

In [4]:
#nothing
x_db = train_db.copy()
y_db = x_db['y']
x_db = x_db.drop('y', axis=1)
X = x_db
y = y_db
res_db = test_db.copy()
res_db = res_db.drop('ID', axis=1)
x_db = x_db.append(res_db).reset_index()
# x_db

In [5]:
def m_hash(db):
    hash_db = pd.DataFrame(index=x_db.index.values,columns=x_db.columns.values)
    for col in db:
        if isinstance(db[col][0],str) is False:
            hash_db[col] = db[col]
        else:
            for i in range(10):
                hash_db[col] = [abs(hash(el))%(10**i) for el in db[col]]
                if (len(set(hash_db[col]))==len(set(db[col]))):
                    break
            if (len(set(hash_db[col]))!=len(set(db[col]))):
                print(len(set(hash_db[col])),
                      len(set(db[col])),
                      "error of hashing!!")
    return hash_db
hash_train_db = m_hash(x_db)

In [6]:
#NaN -> mean
X = hash_train_db
y = y_db

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp_db = imp.fit_transform(X)

In [7]:
X = imp_db
y = y_db

X_train, X_holdout, y_train, y_holdout = train_test_split(X[:50000],
                                                          y,
                                                          test_size=0.1, random_state=0)

In [8]:
clf = DecisionTreeClassifier(max_depth=7, random_state=0)

In [9]:
clf.fit(X_train, y_train)
clf_pred = clf.predict(X_holdout)
print(accuracy_score(y_holdout, clf_pred))
clf_pred = clf.predict(X_train)
print(accuracy_score(y_train, clf_pred))
res = clf.predict(X[50000:])

0.739
0.749


In [10]:
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data, filled=True, rounded=True,
                special_characters=True)

In [11]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

with open("tree.png", "wb") as png:
    png.write(graph.create_png())

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.980989 to fit



In [12]:
res_d = pd.DataFrame(data={'id':range(len(res)),'y':res})
print(res)
res_d.to_csv('res.csv', index=False)
pd.read_csv('res.csv')

[6 4 4 ... 3 4 5]


,id,y
0,0,6
1,1,4
2,2,4
3,3,3
4,4,2
5,5,3
6,6,3
7,7,2
8,8,4
9,9,5
